In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt

from keras import Model, Input, layers

c:\Users\User\miniconda3\envs\paper\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

# load dataset

In [3]:
(ds_train, ds_test), ds_info = tfds.load(
    'cifar10',
    split=['train', 'test'],
    shuffle_files=True,
    with_info=True,
)
print(tf.data.experimental.cardinality(ds_train))
print(tf.data.experimental.cardinality(ds_test))

tf.Tensor(50000, shape=(), dtype=int64)
tf.Tensor(10000, shape=(), dtype=int64)


In [4]:
def normalize_img(img, label):
    return tf.cast(img, tf.float32)/255., label

def apply_normalize(ds, is_test=False, batch_size=16):
    ds = ds.map(
        normalize_img,
        num_parallel_calls=1
    )
    ds = ds.batch(batch_size)
    if not is_test:
        ds = ds.repeat()
        ds = ds.shuffle(200)
    ds = ds.prefetch(tf.data.experimental.AUTOTUNE)
    return ds

# AlexNet

In [48]:
# bias 바꿔줘야 함
class AlexNet(Model):
    def __init__(self, num_classes, filters=[96, 256, 384, 384, 256], kernels=[11, 5, 3, 3, 3],
                 fc=4096, pool_size=3, first_stride=4, pool_stride=2):
        super(AlexNet, self).__init__()
        initializer = tf.initializers.RandomNormal(mean=0, stddev=0.01)

        self.conv1 = tf.keras.layers.Conv2D(filters[0], kernels[0], strides=first_stride, kernel_initializer=initializer,
                                            #bias_initializer='zeros',
                                            name='conv1')
        self.act1 = tf.keras.layers.Activation('relu', name='relu1')
        self.pool1 = tf.keras.layers.MaxPooling2D(pool_size, strides=pool_stride, name='maxpool1')
        self.drop1 = tf.keras.layers.Dropout(0.5, name='dropout1')
        self.lrn1 = tf.keras.layers.Lambda(lambda x: tf.nn.lrn(x, depth_radius=2, bias=1.0, alpha=2e-05, beta=0.75), name='LRN')

        self.conv2 = tf.keras.layers.Conv2D(filters[1], kernels[1], padding='same', kernel_initializer=initializer,
                                            #bias_initializer='ones',
                                            name='conv2')
        self.act2 = tf.keras.layers.Activation('relu', name='relu2')
        self.pool2 = tf.keras.layers.MaxPooling2D(pool_size, pool_stride, name='maxpool2')

        self.conv3 = tf.keras.layers.Conv2D(filters[2], kernels[2], padding='same', kernel_initializer=initializer,
                                            #bias_initializer='zeros',
                                            name='conv3')
        self.act3 = tf.keras.layers.Activation('relu', name='relu3')
        
        self.conv4 = tf.keras.layers.Conv2D(filters[3], kernels[3], padding='same', kernel_initializer=initializer,
                                            bias_initializer='ones', name='conv4')
        self.act4 = tf.keras.layers.Activation('relu', name='relu4')
        
        self.conv5 = tf.keras.layers.Conv2D(filters[4], kernels[4], padding='same', kernel_initializer=initializer,
                                            #bias_initializer='ones',
                                            name='conv5')
        self.act5 = tf.keras.layers.Activation('relu', name='relu5')
        self.pool3 = tf.keras.layers.MaxPooling2D(pool_size, strides=pool_stride, name='maxpool3')

        self.flatten = tf.keras.layers.Flatten(name='flatten')
        self.fc1 = tf.keras.layers.Dense(fc,
                                         #kernel_initializer=initializer,
                                         name='fc4096_1')
        self.act6 = tf.keras.layers.Activation('relu', name='relu6')
        self.drop2 = tf.keras.layers.Dropout(0.5, name='dropout2')

        self.fc2 = tf.keras.layers.Dense(fc,
                                         #kernel_initializer=initializer,
                                         name='fc4096_2')
        self.act7 = tf.keras.layers.Activation('relu', name='relu7')
        self.drop3 = tf.keras.layers.Dropout(0.5, name='dropout3')

        self.fc3 = tf.keras.layers.Dense(num_classes, kernel_initializer=initializer, name='fc_num_classes')

    def call(self, x):
        x = self.conv1(x)
        x = self.act1(x)
        x = self.pool1(x)
        x = self.drop1(x)
        x = self.lrn1(x)

        x = self.conv2(x)
        x = self.act2(x)
        x = self.pool2(x)

        x = self.conv3(x)
        x = self.act3(x)
        x = self.conv4(x)
        x = self.act4(x)
        x = self.conv5(x)
        x = self.act5(x)
        x = self.pool3(x)

        x = self.flatten(x)
        x = self.fc1(x)
        x = self.act6(x)
        x = self.drop2(x)

        x = self.fc2(x)
        x = self.act7(x)
        x = self.drop3(x)

        x = self.fc3(x)
        return x
    def get_summary(self, input_shape=(227, 227, 3)):
        inputs = Input(input_shape)
        return Model(inputs, self.call(inputs)).summary()

In [49]:
# Original AlexNet
AlexNet(1000).get_summary()
# CIFAR-10 적용시 ImageNet과 크기가 달라서 negative dimension 발생 -> 차원 수정
# 예시
# AlexNet(10, kernels=[4,5,3,3,3], fc=2048 ,first_stride=3, pool_size=2, pool_stride=2).get_summary(input_shape=(32, 32, 3))

Model: "model_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_8 (InputLayer)        [(None, 227, 227, 3)]     0         
                                                                 
 conv1 (Conv2D)              (None, 55, 55, 96)        34944     
                                                                 
 relu1 (Activation)          (None, 55, 55, 96)        0         
                                                                 
 maxpool1 (MaxPooling2D)     (None, 27, 27, 96)        0         
                                                                 
 dropout1 (Dropout)          (None, 27, 27, 96)        0         
                                                                 
 LRN (Lambda)                (None, 27, 27, 96)        0         
                                                                 
 conv2 (Conv2D)              (None, 27, 27, 256)       6146

# Train

In [64]:
model = AlexNet(10, filters=[48, 96, 192, 192, 96], kernels=[9, 5, 3, 3, 3], 
                fc=1024,first_stride=2, pool_size=2, pool_stride=2)
model.compile(loss='sparse_crossentropy')
# Compile the model
optimizer = tf.keras.optimizers.SGD(learning_rate=0.001, momentum=0.9)
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metrics = ['accuracy']

model.compile(optimizer=optimizer, loss=loss_fn, metrics=metrics)

In [66]:
BATCH_SIZE = 128
EPOCH = 90

In [61]:
(ds_train, ds_test), ds_info = tfds.load(
    'cifar10',
    split=['train', 'test'],
    as_supervised=True,
    shuffle_files=True,
    with_info=True,
)
ds_train = apply_normalize(ds_train, batch_size=BATCH_SIZE)
ds_test = apply_normalize(ds_test, batch_size=BATCH_SIZE)
history = model.fit(
    ds_train,
    steps_per_epoch=int(ds_info.splits['train'].num_examples/BATCH_SIZE),
    validation_steps=int(ds_info.splits['test'].num_examples/BATCH_SIZE),
    epochs=EPOCH,
    validation_data=ds_test,
    verbose=1,)

Epoch 1/90
390/390 [==============================] - 4s 7ms/step - loss: 2.3032 - accuracy: 0.0987 - val_loss: 2.3026 - val_accuracy: 0.1010
Epoch 2/90
390/390 [==============================] - 3s 7ms/step - loss: 2.3030 - accuracy: 0.0995 - val_loss: 2.3027 - val_accuracy: 0.0975
Epoch 3/90
390/390 [==============================] - 3s 7ms/step - loss: 2.3027 - accuracy: 0.0999 - val_loss: 2.3026 - val_accuracy: 0.1004
Epoch 4/90
390/390 [==============================] - 3s 7ms/step - loss: 2.3027 - accuracy: 0.0976 - val_loss: 2.3026 - val_accuracy: 0.0973
Epoch 5/90
390/390 [==============================] - 2s 6ms/step - loss: 2.3027 - accuracy: 0.0998 - val_loss: 2.3026 - val_accuracy: 0.0964
Epoch 6/90
390/390 [==============================] - 2s 6ms/step - loss: 2.3027 - accuracy: 0.0986 - val_loss: 2.3026 - val_accuracy: 0.0966
Epoch 7/90
386/390 [============================>.] - ETA: 0s - loss: 2.3026 - accuracy: 0.0980

KeyboardInterrupt: 

# visualize

In [37]:
plt.plot(history.history['loss'], 'r')
plt.title('Model training loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['AlexNet'], loc='upper left')
plt.show()

NameError: name 'history' is not defined

In [ ]:
plt.plot(history.history['val_accuracy'], 'b')
plt.title('Model validation accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['AlexNet'], loc='upper left')
plt.show()

# Metric

추가할점
* 이미지 augmentation ,imagenet?
* Metric - top5, top1 추가
* logging - logging하는법 추가 -> .py로 학습
* 학습된 convlayer 시각화